In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from numpy import prod
import datetime
import pandas

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

sc = spark.sparkContext

from pyspark.streaming import StreamingContext

streamContext = StreamingContext(sc,1)

lines = streamContext.socketTextStream("localhost", 9000)

def my_processing(rdd):
    my_lines = rdd.map(lambda l: l.split(","))\
    .map(lambda p: Row(time=p[0],name=p[1], mpg=p[2], cyl=p[3], \
                       disp=p[4], hp=p[5], drat=p[6], wt=p[7], qsec=p[8], vs=p[9],\
                      am=p[10], gear=p[11], carb=p[12]))
    mtcars = my_lines.toDF(["am","carb","cyl","disp","drat","gear","hp","mpg","name","qsec","time","vs","wt"])
    mtcars = mtcars.withColumn('mpg_per_hp', mtcars.mpg * mtcars.hp )
    print(mtcars.show())
    print('############################################################################')
    
lines.foreachRDD(my_processing)

streamContext.start()

+---+----+---+-----+----+----+---+----+-----------------+-----+--------+---+----+----------+
| am|carb|cyl| disp|drat|gear| hp| mpg|             name| qsec|    time| vs|  wt|mpg_per_hp|
+---+----+---+-----+----+----+---+----+-----------------+-----+--------+---+----+----------+
|  1|   6|  6|  145|3.62|   5|175|19.7|     Ferrari Dino| 15.5|10:33:48|  0|2.77|    3447.5|
|  0|   2|  8|  360|3.15|   3|175|18.7|Hornet Sportabout|17.02|10:33:48|  0|3.44|    3272.5|
|  1|   2|  4|120.3|4.43|   5| 91|  26|    Porsche 914-2| 16.7|10:33:48|  0|2.14|    2366.0|
+---+----+---+-----+----+----+---+----+-----------------+-----+--------+---+----+----------+

None
############################################################################
+---+----+---+----+----+----+---+----+-------------------+-----+--------+---+-----+----------+
| am|carb|cyl|disp|drat|gear| hp| mpg|               name| qsec|    time| vs|   wt|mpg_per_hp|
+---+----+---+----+----+----+---+----+-------------------+-----+--------+---